### Libraries

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import date, datetime, timedelta

from tqdm.notebook import tqdm

import tensorflow_hub as hub
from scipy.spatial.distance import cdist

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

##### Days

In [2]:
days=pd.read_csv("days.csv")
days=days.drop(columns="Unnamed: 0")

days = days[days["description"].notnull()]
days

,Date,Name,Country,Year,Month,Day,description,image
0,0009-09-01,Battle of the Teutoburg Forest,NaN,9,9,1,"Allied Germanic peoples, possibly including t...",http://commons.wikimedia.org/wiki/Special:File...
1,0015-01-01,Battle of the Bridges,NaN,15,1,1,Coalition intervention Naval operations Air c...,http://commons.wikimedia.org/wiki/Special:File...
2,0016-01-01,Battle of the Weser River,NaN,16,1,1,Gallic Wars (58 BC – 57 BC) Clades Lolliana (1...,http://commons.wikimedia.org/wiki/Special:File...
3,0030-04-07,Pilate's court,NaN,30,4,7,"In the canonical gospels, Pilate's court refer...",http://commons.wikimedia.org/wiki/Special:File...
4,0043-01-01,Battle of the Medway,NaN,43,1,1,"The Battle of the Medway took place in 43 AD,...",http://commons.wikimedia.org/wiki/Special:File...
...,...,...,...,...,...,...,...,...
55977,1955-06-05 00:00:00,Hervé This,NaN,1955,6,5,birthday,NaN
55978,1921-01-01 00:00:00,Charles Le Gai Eaton,NaN,1921,1,1,birthday,NaN
55979,1913-09-19 00:00:00,Frances Farmer,NaN,1913,9,19,birthday,NaN
55980,1936-12-29 00:00:00,Mary Tyler Moore,NaN,1936,12,29,birthday,NaN


##### Books

In [3]:
books=pd.read_csv("best_books.csv")
books=books.drop(columns="Unnamed: 0")

books

,id,title,authorName,rating,description,rating_count,review_count,img_url,url,title_language
0,3,Harry Potter and the Sorcerer's Stone,J.K. Rowling,4.48,Harry Potter s life is miserable His parents...,8379630,132348,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/3.Harry_Po...,en
1,43504,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.48,Harry Potter s life is miserable His parents...,8379055,132336,https://d15be2nos83ntc.cloudfront.net/images/n...,https://www.goodreads.com/book/show/43504.Harr...,en
2,49772,Harri Potter a Maen yr Athronydd,J.K. Rowling,4.48,Rescued from the outrageous neglect of his au...,8378971,132333,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/49772.Harr...,en
3,2767052,The Hunger Games,Suzanne Collins,4.32,Could you survive on your own in the wild w...,7288063,183095,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/2767052-th...,en
4,12024,Twilight,Stephenie Meyer,3.62,About three things I was absolutely positive ...,5667779,113710,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/12024.Twil...,en
...,...,...,...,...,...,...,...,...,...,...
44458,3401778,The Book on Leadership,John F. MacArthur Jr.,4.14,In The Book on Leadership best selling aut...,1000,89,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/3401778-th...,en
44459,153664,The Scavenger's Guide to Haute Cuisine,Steven Rinella,4.35,Describes one man s efforts to live on the la...,1000,99,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/153664.The...,en
44460,421398,China Marine: An Infantryman's Life After Worl...,Eugene B. Sledge,4.25,See E B Sledge s story in the HBO miniserie...,1000,76,https://i.gr-assets.com/images/S/compressed.ph...,https://www.goodreads.com/book/show/421398.Chi...,en
44461,38757,The Price,Arthur Miller,3.82,In a building slated for imminent demolition ...,1000,91,https://d15be2nos83ntc.cloudfront.net/images/n...,https://www.goodreads.com/book/show/38757.The_...,en


# Model

##### universal-sentence-encoder ****

###### TMDB API + Goodreads

---

In [12]:
#### code takes +- 2 hours ######

movies=pd.read_csv("TMDB_movies_final.csv")
movies = movies.rename(columns={"overview": "description"})

#today = datetime.today()
# today = datetime.today() + timedelta(days=11) #--- use if you want to decide the day

#days = days[(days["Month"] == today.month) & (days["Day"] == today.day)].reset_index(drop=True)

# Start time
start_time = time.time()

# Model
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Function to embed text
def embed_text(text):
    if isinstance(text, str) and text.strip():  # Check if the text is not empty
        return model([text]).numpy()
    else:
        return None

print("Encoding movie descriptions...")
movies['vec'] = [embed_text(desc) for desc in movies['description']]
print("Encoding book descriptions...")
books['vec'] = [embed_text(desc) for desc in books['description']]
    
# Starting the matches file
matches = days.copy()

# Opening the matches lists
match_movie1 = []
match_movie1_descr = []
match_movie2 = []
match_movie2_descr = []
match_movie3 = []
match_movie3_descr = []
match_book1 = []
match_book1_descr = []
match_book2 = []
match_book2_descr = []
match_book3 = []
match_book3_descr = []

for index, day in matches.iterrows():
    if day["description"] == "birthday":
        matched_books = books[books['authorName'] == day['Name']]

        # lets check if its an author or an actor
        if len(matched_books) >= 3:
            # Randomly select 3 books
            matched_books = matched_books.sample(3)

            # Append the book information to the respective lists
            match_book1.append(matched_books.iloc[0]['title'])
            match_book1_descr.append(matched_books.iloc[0]['description'])
            match_book2.append(matched_books.iloc[1]['title'])
            match_book2_descr.append(matched_books.iloc[1]['description'])
            match_book3.append(matched_books.iloc[2]['title'])
            match_book3_descr.append(matched_books.iloc[2]['description'])
            match_movie1.append(None)
            match_movie1_descr.append(None)
            match_movie2.append(None)
            match_movie2_descr.append(None)
            match_movie3.append(None)
            match_movie3_descr.append(None)

        # If there are matched books but less than 3
        elif len(matched_books) > 0:
            match_book1.append(matched_books.iloc[0]['title'])  # Select just one book
            match_book1_descr.append(matched_books.iloc[0]['description'])  # Select just one book
            match_book2.append(None)  # Append None as there is no second book
            match_book2_descr.append(None)  # Append None as there is no second book
            match_book3.append(None)  # Append None as there is no third book
            match_book3_descr.append(None)  # Append None as there is no third book
            match_movie1.append(None)
            match_movie1_descr.append(None)
            match_movie2.append(None)
            match_movie2_descr.append(None)
            match_movie3.append(None)
            match_movie3_descr.append(None)

        # If there are no matched books
        else:
            match_book1.append(None)  # Append None as there is no book
            match_book1_descr.append(None)  # Append None as there is no book
            match_book2.append(None)  # Append None as there is no book
            match_book2_descr.append(None)  # Append None as there is no book
            match_book3.append(None)  # Append None as there is no book
            match_book3_descr.append(None)  # Append None as there is no book

            # And lets go to actors

            matched_movies = movies[movies['actors'].fillna('').str.contains(day['Name'])]

            if len(matched_movies) >= 3:
                # Randomly select 3 movies
                matched_movies = matched_movies.sample(3)

                # Append the movie information to the respective lists
                match_movie1.append(matched_movies.iloc[0]['title'])
                match_movie1_descr.append(matched_movies.iloc[0]['description'])
                match_movie2.append(matched_movies.iloc[1]['title'])
                match_movie2_descr.append(matched_movies.iloc[1]['description'])
                match_movie3.append(matched_movies.iloc[2]['title'])
                match_movie3_descr.append(matched_movies.iloc[2]['description'])

            # If there are matched movies but less than 3
            elif len(matched_movies) > 0:
                match_movie1.append(matched_movies.iloc[0]['title'])  # Select just one movie
                match_movie1_descr.append(matched_movies.iloc[0]['description'])  # Select just one movie
                match_movie2.append(None)  # Append None as there is no second movie
                match_movie2_descr.append(None)  # Append None as there is no second movie
                match_movie3.append(None)  # Append None as there is no third movie
                match_movie3_descr.append(None)  # Append None as there is no third movie

            # If there are no matched movies
            else:
                match_movie1.append(None)  # Append None as there is no movie
                match_movie1_descr.append(None)  # Append None as there is no movie
                match_movie2.append(None)  # Append None as there is no movie
                match_movie2_descr.append(None)  # Append None as there is no movie
                match_movie3.append(None)  # Append None as there is no movie
                match_movie3_descr.append(None)  # Append None as there is no movie

    else:
        day_text = day['Name'] if pd.isna(day['description']) else day['Name'] + " " + day['description']
        day_vec = embed_text(day_text)

        movies['similarity'] = [1 - cdist(vec, day_vec, 'cosine')[0][0] if vec is not None else np.nan for vec in
                                movies['vec']]
        books['similarity'] = [1 - cdist(vec, day_vec, 'cosine')[0][0] if vec is not None else np.nan for vec in
                               books['vec']]

        similar_movies = movies.sort_values(by='similarity', ascending=False)
        similar_books = books.sort_values(by='similarity', ascending=False)

        match_movie1.append(similar_movies['title'].iloc[0])
        match_movie1_descr.append(similar_movies['description'].iloc[0])
        match_movie2.append(similar_movies['title'].iloc[1])
        match_movie2_descr.append(similar_movies['description'].iloc[1])
        match_movie3.append(similar_movies['title'].iloc[2])
        match_movie3_descr.append(similar_movies['description'].iloc[2])
        match_book1.append(similar_books['title'].iloc[0])
        match_book1_descr.append(similar_books['description'].iloc[0])
        match_book2.append(similar_books['title'].iloc[1])
        match_book2_descr.append(similar_books['description'].iloc[1])
        match_book3.append(similar_books['title'].iloc[2])
        match_book3_descr.append(similar_books['description'].iloc[2])

    print(day['Name'])

matches["match_movie1"] = match_movie1
matches["match_movie1_descr"] = match_movie1_descr
matches["match_movie2"] = match_movie2
matches["match_movie2_descr"] = match_movie2_descr
matches["match_movie3"] = match_movie3
matches["match_movie3_descr"] = match_movie3_descr
matches["match_book1"] = match_book1
matches["match_book1_descr"] = match_book1_descr
matches["match_book2"] = match_book2
matches["match_book2_descr"] = match_book2_descr
matches["match_book3"] = match_book3
matches["match_book3_descr"] = match_book3_descr

matches = matches.dropna(subset=['match_movie1', 'match_book1'], how='all').reset_index(drop=True)
matches.drop_duplicates(subset=['description', 'match_movie1'], inplace=True)

matches.to_csv("all_matches.csv", index=False)

# End time
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

matches

Encoding movie descriptions...
Encoding book descriptions...
Battle of the Teutoburg Forest
Battle of the Bridges
Battle of the Weser River
Pilate's court
Battle of the Medway
Council of Jerusalem
Roman conquest of Anglesey
Roman conquest of Anglesey
Battle of Watling Street
Battle of Watling Street
Jerusalem riots of 66
Zealot Temple siege
Battle of Mons Graupius
Battle of Mons Graupius
Battle of Ikh Bayan
Battle of Xiangyang
Battle of Cyzicus
Battle of Lugdunum
Battle of Guandu
Battle of Bowang
Battle of Red Cliffs
Battle of Changban
Battle of Xiaoting
battle of Hormizdegan
Battle at the Harzhorn
Battle of Misiche
Battle of Abritus
Siege of Antioch
Battle of Mediolanum
battle of Edessa
262 Southwest Anatolia earthquake
Battle of Lake Benacus
Battle of Naissus
Battle of Pavia
Crisis of the Third Century
Battle of the Milvian Bridge
Battle of the Milvian Bridge
Battle of the Hellespont
Battle of Adrianople
Battle of Adrianople
Battle of Chrysopolis
First Council of Nicaea
Battle of Mon

Mongol siege of Kaifeng
Siege of Burriana
Mongol conquest of the Song Dynasty
Siege of Constantinople (1235)
Siege of Ryazan
Siege of Kolomna
Mongol invasion of Kievan Rus'
Siege of Kozelsk
Livonian campaign against Rus'
Siege of Montségur
Seventh Crusade
Siege of Seville
Mongol campaign against the Nizaris
Mongol campaign against the Nizaris
War of the Euboeote Succession
Venetian–Genoese Wars
War of Saint Sabas
Second Mongol invasion of Poland
Siege of Constantinople (1260)
Battle of Ain Jalut
Berke–Hulagu war
Siege of Königsberg
Second Barons' War
Mudéjar revolt of 1264–66
Mise of Amiens
Byzantine–Ottoman Wars
Siege of Kenilworth
Eighth Crusade
Ninth Crusade
Uprising of Ivaylo
First Mongol invasion of Burma
Sicilian Vespers
Lithuanian Crusade
Battle of Meloria
Third Mongol invasion of Poland
Fall of Tripoli
Siege of Acre (1291)
Treaty of Alcañices
Battle of the Golden Spurs
Fall of Ruad
Battle of the Golden Spurs
Battle of Zierikzee
Battle of Zierikzee
First Siege of Gibraltar
Battl

Siege of Carrigafoyle Castle
Siege of Carrigafoyle Castle
War of the Portuguese Succession
Russian conquest of Siberia
Union of Utrecht
Battle of Glenmalure
Siege of Smerwick
Battle of Salga
Battle of Noordhorn
Siege of Niezijl
Siege of Takamatsu
Siege of Pskov
Battle of Tenmokuzan
Honnō-ji Incident
Battle of Yamazaki
Battle of Ponta Delgada
Battle of Ponta Delgada
Battle of Chuvash Cape
Destruction of the Oberstift
Cologne War
Siege of Godesberg
French Fury
Battle of São Vicente
Battle of Torches
Battle of Shizugatake
Cuncolim Revolt
Battle of Komaki and Nagakute
Fall of Antwerp
Siege of Ghent (1583–1584)
Battle of Werl
Siege of IJsseloord
Battle of the Western Isles
Babington Plot
Battle of Allt Camhna
Battle of Leckmelm
Battle of Boksum
1586 Tenshō earthquake
Battle of Cartagena de Indias
Capture of Axel
Destruction of Neuss
Battle of Zutphen
Singeing the King of Spain's Beard
Siege of Sluis (1587)
Battle of Coutras
Battle of Byczyna
War of the Polish Succession
Day of the Barricade

Jacobite rising of 1689
Siege of Carrickfergus
1689 Boston revolt
Battle of Bantry Bay
Siege of Derry
Treaty of Nerchinsk
Battle of Cromdale
Battle of the Boyne
Siege of Athlone
Battle of Slankamen
Battle of Leuze
Williamite War in Ireland
massacre of Glencoe
Battles of Barfleur and La Hougue
Battles of Barfleur and La Hougue
1693 Sicily earthquake
Battle of Lagos
Battle of Landen
Battle of Camaret
Battle of Texel (1694)
Capitulation of Diksmuide
Bombardment of Brussels
Siege of Fort Nashwaak
Battle of Fundy Bay
1697 Polish–Lithuanian royal election
Battle of Hudson's Bay
Battle of Zenta
Streltsy Uprising
Siege of Fort Jesus
Scanian War
Russo-Turkish War (1768–1774)
War of the Spanish Succession
Battle of Carpi
Crossing of the Düna
Crossing of the Düna
Battle of Chiari
Battle of Flint River
Battle of Cremona
Siege of Kaiserswerth
Battle of Klissow
Battle of Luzzara
Action of August 1702
Newfoundland expedition
Battle of Friedlingen
Battle of Vigo Bay
Siege of St. Augustine
Great Storm 

Battle of Nassau
Battle of Moore's Creek Bridge
Battle of Saint-Pierre
Battle of Block Island
Battle of Trois-Rivières
Battle of Sullivan's Island
Battle of Turtle Gut Inlet
First Battle of Terrenate
Battle of Long Island
Battle of Long Island
Landing at Kip's Bay
Battle of Harlem Heights
Great Fire of New York
Battle of Valcour Island
Battle of Valcour Island
Battle of Valcour Island
Battle of Valcour Island
Battle of Pell's Point
Battle of White Plains
Battle of Fort Washington
Battle of Fort Washington
Washington's crossing of the Delaware River
Ambush of Geary
Battle of Trenton
Siege of Fort Stanwix
Battle of White Marsh
Battle of the Assunpink Creek
Battle of Princeton
Battle of Millstone
Battle of Bound Brook
Battle of Ridgefield
Battle of Short Hills
Battle of Hubbardton
Battle of Fort Anne
Battle of Oriskany
Battle of Oriskany
Battle of Oriskany
Battle of Bennington
Battle of Staten Island
Siege of Fort Henry (1777)
Battle of Cooch's Bridge
Battle of Brandywine
Battle of German

...

Richard Rorty
Grace Paley
John Masefield
Edward E. Kramer
Yasushi Inoue
Robert Hellenga
Suze Rotolo
Sven Nordqvist
Norah Lofts
Shirley Hazzard
Ruth Benedict
Robin Lee Graham
Bill Hicks
Scott Rosenberg
John  Peel
Henry Louis Gates Jr.
Kay Arthur
Melody Carlson
Mark   Fisher
Ève Curie
Hal Higdon
Gerard Manley Hopkins
William Bligh
Karin Alvtegen
Candace Camp
Oakley Hall
Alex Grey
Phil Town
Marianne Moore
Benedicta Ward
Damon Galgut
Hugo Claus
Peter Kay
Tim Willocks
Laurence J. Peter
Bhikkhu Bodhi
Devdutt Pattanaik
Aaron Elkins
Janet Burroway
Michelle West
Martin Caidin
William Shatner
Rollo May
Gavin Maxwell
Theodore Roosevelt
Rem Koolhaas
Ingeborg Bachmann
Haing Ngor
Betsy Byars
Jo Edwards
Douglas E. Winter
Kaare Andrews
Hart Crane
Jean Fritz
Brian Haig
Sloan Wilson
Robb White
Maureen Jennings
Stephanie Coontz
John Barrowman
Elena Poniatowska
Alex Ross
Alice McDermott
Thomas Gilovich
Mona Golabek
Peggy Noonan
Andre Dubus
Vaughan Roberts
R.F. Delderfield
Margaret Campbell Barnes
Nadeem A

Daniel Alarcón
Alex Archer
Jeremy Paxman
Godfried Bomans
Lu Xun
Willis Barnstone
Arlene Blum
Hakim Bey
Peter Lamborn Wilson
Edward Bunker
Harvey Fierstein
Raimond Gaita
Suzanne Finstad
Dan Eldon
Will Cuppy
John Julius Norwich
John P. Marquand
Raymond Arroyo
Michael Ian Black
Drew Carey
Philippe Djian
Charlie Kaufman
Mike Krzyzewski
Chris Bunch
Kenneth Cook
Nicola Barker
Aleksey Nikolayevich Tolstoy
Andrew S. Tanenbaum
Monica Sjöö
Li Zhisui
Yoko Ono
Suzy McKee Charnas
Bernard Goldberg
Nigel C. Benson
Joshua Braff
Eminem
W.R. Burnett
Paul Hawken
John Braine
Conrad Black
Alice Borchardt
David R. Roediger
Amanda Cross
Richard McKenna
Jack Gilbert
Homi K. Bhabha
Parley P. Pratt
Leonard Wibberley
P.T. Barnum
Howard Gardner
C.K. Prahalad
Ray Manzarek
Kristin Harmel
Olivia Manning
Michael    Maloney
James Kochalka
Naomi Mitchison
Kyle Mills
Ying Chang Compestine
David Graeber
Carol Ryrie Brink
Douglas Southall Freeman
Sam Greenlee
Robert Coover
Shirley Temple Black
G.M. Ford
David J. Chalmers


Flora Annie Steel
Burton Watson
Maria Edgeworth
Mark Doty
Louise Brooks
Harry Patterson
Noel Fielding
Roxana Robinson
Rodney Stark
Stephen McCauley
Klas Östergren
George Meredith
William Blum
Clive James
Leonard Michaels
Freya North
Jake Arnott
Novalis
Colin  Ward
Terry Bisson
Desmond Bagley
Thomas Gifford
Taro Yashima
Carolyn Treffinger
Janet Dailey
Robert Aickman
Antoine Bello
Diet Eman
Jim Henson
Judith Hermann
Trevor Hastie
Laurie Graham
Leo Frankowski
Albert Ellis
Edward Mead Earle
Richard Appignanesi
Peter Paret
Mark Driscoll
Frank Deford
Kim   Wilson
Patrick McDonnell
Janet McNaughton
James Hamilton-Paterson
Pamela Morsi
Gerard Jones
Stewart Brand
Tracy Hickman
Alan Aldridge
Ernst Neufert
Fergus Henderson
Jeanne Marie Laskas
Gus Hansen
Bill Mollison
Angelina Jolie
Donna J. Haraway
Gary Schroen
Paul A. Offit
Bear Grylls
Charles Godfrey Leland
Mary Hunter Austin
Sri Sri Ravi Shankar
Thornton W. Burgess
Nina Kiriki Hoffman
Barry Crump
Gina Cerminara
Joy Harjo
Robert Creeley
Ephraim

Linda Greenhouse
Lucius Shepard
Sohrab Sepehri
Ismat Chughtai
Lin Yutang
Franz Joseph
Victor Serge
Tara June Winch
David G. Myers
Kim Addonizio
Marcel Pagnol
Fiona Walker
David J. Skal
Stephen    Lewis
Margaret  Wilson
Robert Sobel
Dorothy Porter
Susan Griffin
Robert C. Martin
Sheree Renée Thomas
Kurt Andersen
David Bischoff
James Sallis
Seán O'Casey
Parke Godwin
Phyllis A. Whitney
Bernice Rubens
John Cleese
George Smoot
P.J. Haarsma
Michael Baden
Norman Lebrecht
Edna Lewis
Robin Morgan
Michael D. Coe
Eric  Williams
David Bohm
Napoleon A. Chagnon
Samit Basu
Lidia Matticchio Bastianich
Giacomo Leopardi
Celeste De Blasis
Boy George
Hal Foster
James N. Rowe
Sierra St. James
Jennifer Storm
Daniel Carter Beard
Chuck Norris
Miroslav Krleža
Mantak Chia
Anita Rau Badami
Gail Evans
Owen Chase
Maurice Gee
Daniel Estulin
Jeffrey Ford
Robert Shearman
Willie Nelson
Joan Nestle
Rich Cohen
Christopher Anvil
Robert Kagan
Albert M. Wolters
Tom Keneally
Marie Hall Ets
Nancy Garden
Steve Englehart
Daniel

,Date,Name,Country,Year,Month,Day,description,image,match_movie1,match_movie1_descr,match_movie2,match_movie2_descr,match_movie3,match_movie3_descr,match_book1,match_book1_descr,match_book2,match_book2_descr,match_book3,match_book3_descr
0,0009-09-01,Battle of the Teutoburg Forest,NaN,9,9,1,"Allied Germanic peoples, possibly including t...",http://commons.wikimedia.org/wiki/Special:File...,King Arthur,"The story of the Arthurian legend, based on th...",The Lost Battalion,Fact-based war drama about an American battali...,The Glory Guys,Though a fictionalized Western based on George...,"Armageddon: The Battle for Germany, 1944-1945",In September 1944 the Allies believed that H...,The Punic Wars,An impressive new historian of Roman warfare ...,A History of Warfare,The acclaimed author of The Face of Battle ex...
1,0015-01-01,Battle of the Bridges,NaN,15,1,1,Coalition intervention Naval operations Air c...,http://commons.wikimedia.org/wiki/Special:File...,The Gallant Hours,A semi-documentary dramatization of five weeks...,Away All Boats,"The story of USS 'Belinda', a U.S. naval ship,...",A Bridge Too Far,"Operation Market Garden, September 1944. The A...",A Bridge Too Far,The classic account of one of the most dramat...,Miracle at Midway,The Battle of Midway was perhaps the most sig...,The Last Stand Of The Tin Can Sailors,On the morning of October 25 1944 a pivotal...
2,0016-01-01,Battle of the Weser River,NaN,16,1,1,Gallic Wars (58 BC – 57 BC) Clades Lolliana (1...,http://commons.wikimedia.org/wiki/Special:File...,The Last Legion,"As the Roman empire crumbles, young Romulus Au...",Morituris: Legions of the Dead,"Rome, in 73 B.C. Spartacus led the rebellion o...",Amazons and Gladiators,"During the Roman Empire, General Crassius rule...","The History of Rome, Books 1-5: The Rise of Rome",The fates ordained the founding of this gre...,The Punic Wars,An impressive new historian of Roman warfare ...,"The History of Rome, Books 1-5: The Early Hist...",Livy c 59 BC AD 17 dedicated most of his l...
3,0030-04-07,Pilate's court,NaN,30,4,7,"In the canonical gospels, Pilate's court refer...",http://commons.wikimedia.org/wiki/Special:File...,The Greatest Story Ever Told,From his birth in Bethlehem to his death and e...,Thomas,"When Jesus is taken off the cross at Golgotha,...",The Body,An ancient skeleton has been discovered in Jer...,Works of Josephus,Encounter the world and traditions that Jesu...,The Works of Josephus,Encounter the world and traditions that Jesu...,The Last Week: A Day-by-Day Account of Jesus's...,Top Jesus scholars Marcus J Borg and John Do...
4,0043-01-01,Battle of the Medway,NaN,43,1,1,"The Battle of the Medway took place in 43 AD,...",http://commons.wikimedia.org/wiki/Special:File...,The Viking Queen,"Set in ancient Britain, at a time when much of...",The Horsemen,"In the poor, desolate northern provinces of th...",Morituris: Legions of the Dead,"Rome, in 73 B.C. Spartacus led the rebellion o...",The Eagle and the Raven,Spanning three generations this historical n...,When the Eagle Hunts,Well paced and intricately plotted When th...,Scipio Africanus: Greater than Napoleon,Scipio Africanus 236 183 B C was one of th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33656,1973-07-08 00:00:00,Sally Lindsay,NaN,1973,7,8,birthday,NaN,Peter Kay: Live at the Bolton Albert Halls,In Live at the Bolton Albert Halls Peter Kay p...,None,None,None,None,None,None,None,None,None,None
33663,1920-02-03 00:00:00,Russell Arms,NaN,1920,2,3,birthday,NaN,Captains of the Clouds,"Inspired by Churchill's Dunkirk speech, brash,...",By the Light of the Silvery Moon,Marjorie Winfield's engagement to Bill Sherman...,High Wall,"Steven Kenet, suffering from a recurring brain...",None,None,None,None,None,None
33665,1982-03-12 00:00:00,Samm Levine,NaN,1982,3,12,birthday,NaN,Ollie Klublershturf vs. the Nazis,An off-the-wall comedy set around the dinner t...,The Last Blockbuster,A documentary on the last remaining Bl

###### IMDB webscrap + Goodreads

In [13]:
#### code takes +- 2 hours ######

movies=pd.read_csv("imdb_movie_fetch.csv")
movies=movies.drop(columns="Unnamed: 0")

today = datetime.today()
# today = datetime.today() + timedelta(days=10)  # --- use if you want to decide the day

# days = days[(days["Month"] == today.month) & (days["Day"] == today.day)].reset_index(drop=True)

# Start time
start_time = time.time()

# Model
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Function to embed text
def embed_text(text):
    if isinstance(text, str) and text.strip():  # Check if the text is not empty
        return model([text]).numpy()
    else:
        return None

# Starting the matches file
matches2 = days.copy()

# Opening the matches lists
match_movie1 = []
match_movie1_descr = []
match_movie2 = []
match_movie2_descr = []
match_movie3 = []
match_movie3_descr = []
match_book1 = []
match_book1_descr = []
match_book2 = []
match_book2_descr = []
match_book3 = []
match_book3_descr = []

print("Encoding movie descriptions...")
movies['vec'] = [embed_text(desc) for desc in movies['description']]
print("Encoding book descriptions...")
books['vec'] = [embed_text(desc) for desc in books['description']]

for index, day in matches2.iterrows():
    if day["description"] == "birthday":
        matched_books = books[books['authorName'] == day['Name']]

        # lets check if its an author or an actor
        if len(matched_books) >= 3:
            # Randomly select 3 books
            matched_books = matched_books.sample(3)

            # Append the book information to the respective lists
            match_book1.append(matched_books.iloc[0]['title'])
            match_book1_descr.append(matched_books.iloc[0]['description'])
            match_book2.append(matched_books.iloc[1]['title'])
            match_book2_descr.append(matched_books.iloc[1]['description'])
            match_book3.append(matched_books.iloc[2]['title'])
            match_book3_descr.append(matched_books.iloc[2]['description'])
            match_movie1.append(None)
            match_movie1_descr.append(None)
            match_movie2.append(None)
            match_movie2_descr.append(None)
            match_movie3.append(None)
            match_movie3_descr.append(None)

        # If there are matched books but less than 3
        elif len(matched_books) > 0:
            match_book1.append(matched_books.iloc[0]['title'])  # Select just one book
            match_book1_descr.append(matched_books.iloc[0]['description'])  # Select just one book
            match_book2.append(None)  # Append None as there is no second book
            match_book2_descr.append(None)  # Append None as there is no second book
            match_book3.append(None)  # Append None as there is no third book
            match_book3_descr.append(None)  # Append None as there is no third book
            match_movie1.append(None)
            match_movie1_descr.append(None)
            match_movie2.append(None)
            match_movie2_descr.append(None)
            match_movie3.append(None)
            match_movie3_descr.append(None)

        # If there are no matched books
        else:
            match_book1.append(None)  # Append None as there is no book
            match_book1_descr.append(None)  # Append None as there is no book
            match_book2.append(None)  # Append None as there is no book
            match_book2_descr.append(None)  # Append None as there is no book
            match_book3.append(None)  # Append None as there is no book
            match_book3_descr.append(None)  # Append None as there is no book

            # And lets go to actors

            matched_movies = movies[movies['actors'].fillna('').str.contains(day['Name'])]

            if len(matched_movies) >= 3:
                # Randomly select 3 movies
                matched_movies = matched_movies.sample(3)

                # Append the movie information to the respective lists
                match_movie1.append(matched_movies.iloc[0]['title'])
                match_movie1_descr.append(matched_movies.iloc[0]['description'])
                match_movie2.append(matched_movies.iloc[1]['title'])
                match_movie2_descr.append(matched_movies.iloc[1]['description'])
                match_movie3.append(matched_movies.iloc[2]['title'])
                match_movie3_descr.append(matched_movies.iloc[2]['description'])

            # If there are matched movies but less than 3
            elif len(matched_movies) > 0:
                match_movie1.append(matched_movies.iloc[0]['title'])  # Select just one movie
                match_movie1_descr.append(matched_movies.iloc[0]['description'])  # Select just one movie
                match_movie2.append(None)  # Append None as there is no second movie
                match_movie2_descr.append(None)  # Append None as there is no second movie
                match_movie3.append(None)  # Append None as there is no third movie
                match_movie3_descr.append(None)  # Append None as there is no third movie

            # If there are no matched movies
            else:
                match_movie1.append(None)  # Append None as there is no movie
                match_movie1_descr.append(None)  # Append None as there is no movie
                match_movie2.append(None)  # Append None as there is no movie
                match_movie2_descr.append(None)  # Append None as there is no movie
                match_movie3.append(None)  # Append None as there is no movie
                match_movie3_descr.append(None)  # Append None as there is no movie

    else:
        day_text = day['Name'] if pd.isna(day['description']) else day['Name'] + " " + day['description']
        day_vec = embed_text(day_text)

        movies['similarity'] = [1 - cdist(vec, day_vec, 'cosine')[0][0] if vec is not None else np.nan for vec in
                                movies['vec']]
        books['similarity'] = [1 - cdist(vec, day_vec, 'cosine')[0][0] if vec is not None else np.nan for vec in
                               books['vec']]

        similar_movies = movies.sort_values(by='similarity', ascending=False)
        similar_books = books.sort_values(by='similarity', ascending=False)

        match_movie1.append(similar_movies['title'].iloc[0])
        match_movie1_descr.append(similar_movies['description'].iloc[0])
        match_movie2.append(similar_movies['title'].iloc[1])
        match_movie2_descr.append(similar_movies['description'].iloc[1])
        match_movie3.append(similar_movies['title'].iloc[2])
        match_movie3_descr.append(similar_movies['description'].iloc[2])
        match_book1.append(similar_books['title'].iloc[0])
        match_book1_descr.append(similar_books['description'].iloc[0])
        match_book2.append(similar_books['title'].iloc[1])
        match_book2_descr.append(similar_books['description'].iloc[1])
        match_book3.append(similar_books['title'].iloc[2])
        match_book3_descr.append(similar_books['description'].iloc[2])

    print(day['Name'])

matches2["match_movie1"] = match_movie1
matches2["match_movie1_descr"] = match_movie1_descr
matches2["match_movie2"] = match_movie2
matches2["match_movie2_descr"] = match_movie2_descr
matches2["match_movie3"] = match_movie3
matches2["match_movie3_descr"] = match_movie3_descr
matches2["match_book1"] = match_book1
matches2["match_book1_descr"] = match_book1_descr
matches2["match_book2"] = match_book2
matches2["match_book2_descr"] = match_book2_descr
matches2["match_book3"] = match_book3
matches2["match_book3_descr"] = match_book3_descr

matches2 = matches2.dropna(subset=['match_movie1', 'match_book1'], how='all').reset_index(drop=True)
matches2.drop_duplicates(subset=['description', 'match_movie1'], inplace=True)

matches2.to_csv("all_matches2.csv", index=False)

# End time
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

matches2

Encoding movie descriptions...
Encoding book descriptions...
Battle of the Teutoburg Forest
Battle of the Bridges
Battle of the Weser River
Pilate's court
Battle of the Medway
Council of Jerusalem
Roman conquest of Anglesey
Roman conquest of Anglesey
Battle of Watling Street
Battle of Watling Street
Jerusalem riots of 66
Zealot Temple siege
Battle of Mons Graupius
Battle of Mons Graupius
Battle of Ikh Bayan
Battle of Xiangyang
Battle of Cyzicus
Battle of Lugdunum
Battle of Guandu
Battle of Bowang
Battle of Red Cliffs
Battle of Changban
Battle of Xiaoting
battle of Hormizdegan
Battle at the Harzhorn
Battle of Misiche
Battle of Abritus
Siege of Antioch
Battle of Mediolanum
battle of Edessa
262 Southwest Anatolia earthquake
Battle of Lake Benacus
Battle of Naissus
Battle of Pavia
Crisis of the Third Century
Battle of the Milvian Bridge
Battle of the Milvian Bridge
Battle of the Hellespont
Battle of Adrianople
Battle of Adrianople
Battle of Chrysopolis
First Council of Nicaea
Battle of Mon

Mongol siege of Kaifeng
Siege of Burriana
Mongol conquest of the Song Dynasty
Siege of Constantinople (1235)
Siege of Ryazan
Siege of Kolomna
Mongol invasion of Kievan Rus'
Siege of Kozelsk
Livonian campaign against Rus'
Siege of Montségur
Seventh Crusade
Siege of Seville
Mongol campaign against the Nizaris
Mongol campaign against the Nizaris
War of the Euboeote Succession
Venetian–Genoese Wars
War of Saint Sabas
Second Mongol invasion of Poland
Siege of Constantinople (1260)
Battle of Ain Jalut
Berke–Hulagu war
Siege of Königsberg
Second Barons' War
Mudéjar revolt of 1264–66
Mise of Amiens
Byzantine–Ottoman Wars
Siege of Kenilworth
Eighth Crusade
Ninth Crusade
Uprising of Ivaylo
First Mongol invasion of Burma
Sicilian Vespers
Lithuanian Crusade
Battle of Meloria
Third Mongol invasion of Poland
Fall of Tripoli
Siege of Acre (1291)
Treaty of Alcañices
Battle of the Golden Spurs
Fall of Ruad
Battle of the Golden Spurs
Battle of Zierikzee
Battle of Zierikzee
First Siege of Gibraltar
Battl

Siege of Carrigafoyle Castle
Siege of Carrigafoyle Castle
War of the Portuguese Succession
Russian conquest of Siberia
Union of Utrecht
Battle of Glenmalure
Siege of Smerwick
Battle of Salga
Battle of Noordhorn
Siege of Niezijl
Siege of Takamatsu
Siege of Pskov
Battle of Tenmokuzan
Honnō-ji Incident
Battle of Yamazaki
Battle of Ponta Delgada
Battle of Ponta Delgada
Battle of Chuvash Cape
Destruction of the Oberstift
Cologne War
Siege of Godesberg
French Fury
Battle of São Vicente
Battle of Torches
Battle of Shizugatake
Cuncolim Revolt
Battle of Komaki and Nagakute
Fall of Antwerp
Siege of Ghent (1583–1584)
Battle of Werl
Siege of IJsseloord
Battle of the Western Isles
Babington Plot
Battle of Allt Camhna
Battle of Leckmelm
Battle of Boksum
1586 Tenshō earthquake
Battle of Cartagena de Indias
Capture of Axel
Destruction of Neuss
Battle of Zutphen
Singeing the King of Spain's Beard
Siege of Sluis (1587)
Battle of Coutras
Battle of Byczyna
War of the Polish Succession
Day of the Barricade

Jacobite rising of 1689
Siege of Carrickfergus
1689 Boston revolt
Battle of Bantry Bay
Siege of Derry
Treaty of Nerchinsk
Battle of Cromdale
Battle of the Boyne
Siege of Athlone
Battle of Slankamen
Battle of Leuze
Williamite War in Ireland
massacre of Glencoe
Battles of Barfleur and La Hougue
Battles of Barfleur and La Hougue
1693 Sicily earthquake
Battle of Lagos
Battle of Landen
Battle of Camaret
Battle of Texel (1694)
Capitulation of Diksmuide
Bombardment of Brussels
Siege of Fort Nashwaak
Battle of Fundy Bay
1697 Polish–Lithuanian royal election
Battle of Hudson's Bay
Battle of Zenta
Streltsy Uprising
Siege of Fort Jesus
Scanian War
Russo-Turkish War (1768–1774)
War of the Spanish Succession
Battle of Carpi
Crossing of the Düna
Crossing of the Düna
Battle of Chiari
Battle of Flint River
Battle of Cremona
Siege of Kaiserswerth
Battle of Klissow
Battle of Luzzara
Action of August 1702
Newfoundland expedition
Battle of Friedlingen
Battle of Vigo Bay
Siege of St. Augustine
Great Storm 

Battle of Nassau
Battle of Moore's Creek Bridge
Battle of Saint-Pierre
Battle of Block Island
Battle of Trois-Rivières
Battle of Sullivan's Island
Battle of Turtle Gut Inlet
First Battle of Terrenate
Battle of Long Island
Battle of Long Island
Landing at Kip's Bay
Battle of Harlem Heights
Great Fire of New York
Battle of Valcour Island
Battle of Valcour Island
Battle of Valcour Island
Battle of Valcour Island
Battle of Pell's Point
Battle of White Plains
Battle of Fort Washington
Battle of Fort Washington
Washington's crossing of the Delaware River
Ambush of Geary
Battle of Trenton
Siege of Fort Stanwix
Battle of White Marsh
Battle of the Assunpink Creek
Battle of Princeton
Battle of Millstone
Battle of Bound Brook
Battle of Ridgefield
Battle of Short Hills
Battle of Hubbardton
Battle of Fort Anne
Battle of Oriskany
Battle of Oriskany
Battle of Oriskany
Battle of Bennington
Battle of Staten Island
Siege of Fort Henry (1777)
Battle of Cooch's Bridge
Battle of Brandywine
Battle of German

...

Dervla Murphy
Pico Iyer
J.P. Moreland
Alfred V. Aho
Li-Young Lee
Madeleine Brent
Stuart Maconie
David Wojnarowicz
Judith Merkle Riley
Diane Arbus
Richard Hofstadter
Maile Meloy
Barry Hannah
Shelagh Delaney
Geoffrey Wellum
Candace Fleming
Andrew Lloyd Webber
Siegfried Lenz
Andrew Marr
Graham Joyce
Stephen Leather
Bryan Talbot
Joseph P. Lash
Raph Koster
Tony Buzan
Henry Wadsworth Longfellow
Ramana Maharshi
Joel Goldman
David Talbot
Jaime Hernández
James McManus
Steven H. Strogatz
Theodore Dalrymple
Philip Sugden
Richard Kadrey
Edward Dolnick
L. Sprague de Camp
Ayi Kwei Armah
Margaret Mead
Shepard Fairey
John Jantsch
M. Ageyev
Tim Parks
Patrick White
Jesse Schell
Wendy Pini
Delia Smith
Connie Briscoe
Wendy Wasserstein
Yoshihiro Togashi
Rebecca Walker
Peter Berresford Ellis
Margaret Drabble
Geronimo
Henri Troyat
Richard J. Herrnstein
D. Martyn Lloyd-Jones
Kirby McCauley
Camille Paglia
Rosa Luxemburg
Gabriela Zapolska
Nelson Algren
Marjorie Pay Hinckley
Paul M. Buhle
Peter Zumthor
Neal Cass

Robert Tine
Steve Austin
Edward Bulwer-Lytton
David Reichert
Mark Strand
Victor Klemperer
Elias Khoury
Tori Amos
Anthony Giddens
Amina Wadud
Robert Paul Weston
Fredric Brown
D.W. Winnicott
Elias Canetti
Garry Kasparov
Bruno Latour
Phil Vischer
Joseph  O'Connor
Renate Dorrestein
Lucy Monroe
Kristine Kathryn Rusch
Jimmy Santiago Baca
Alan M. Dershowitz
Reginald Hudlin
Robert van Gulik
Mikhail Naimy
Cornell Woolrich
Ruth Klüger
Rosemary Rogers
Charna Halpern
John Layman
Donella H. Meadows
Henri Barbusse
Chuck Pfarrer
Geoff Dyer
Jason Hawes
Kurt Aland
James Dale Davidson
Stephen Wolfram
Marc Bloch
Jennifer  Jordan
Ernle Bradford
Adam Zamoyski
Ōgai Mori
Jack Kirby
Nathalie Sarraute
Tim Russert
Catherine Hamlin
Denise Chong
Elizabeth Edmondson
Gail Sheehy
Roger  Deakin
Richard Evelyn Byrd
Kingsley Amis
Courtney Love
Michka Assayas
Christine King Farris
John Jackson Miller
Bob  Greene
Dorothy Bryant
Rosemary Wells
Robert Bauval
David Niven
Edward J. Larson
C.V. Wedgwood
Victoria Beckham
Micol

Jack Trout
Chuck Barris
Larry Crabb
Louise  Brown
Alan Davis
Siddharth Dhanvant Shanghvi
Jennifer Wilde
Gordon Parks
Reay Tannahill
David Plotz
Nick Tosches
Alex  Robinson
Akhil Reed Amar
Aung San Suu Kyi
Susan Kyle
John M. Perkins
Esther  Williams
Eric Bischoff
Josemaría Escrivá
Russell Kirk
Jonathon King
Ian Halperin
Barbara Brooks Wallace
Bernie Sanders
Saul Friedländer
Robert Twigger
Justina Robson
Fernand Braudel
Douglas S. Massey
Josephine Winslow Johnson
Jean Little
Anton Szandor LaVey
Ian Buruma
Peter Balakian
Duane Swierczynski
Cody Lundin
Alfred Kubin
Richard B. Hays
Susan Minot
Trixie Koontz
Ellen Meiksins Wood
Eric Walters
Farah Pahlavi
Martin Duberman
Sam Giancana
Akimitsu Takagi
Derek Parfit
Lanford Wilson
Mary Robison
Emily Brightwell
Thomas Dubay
Elizabeth Cady Stanton
Jean-Pierre Vernant
Amy Carmichael
Alan Burgess
Mary Morris
Elizabeth Wein
Stefan Aust
Peter Rollins
Carol Emshwiller
B.S. Johnson
Katie Price
John Gilstrap
Louie Giglio
Graeme Goldsworthy
Charles Addams


Elvira Lindo
Ramakrishna
Larry Winget
Benoît B. Mandelbrot
Lisa Whelchel
Jarrett J. Krosoczka
John W. Loftus
André Schwarz-Bart
Edgar Wallace
Lana Wachowski
Amir D. Aczel
John M. Frame
Eth Clifford
Donn Pearce
Charlie Connelly
Raffi Cavoukian
Paul McKenna
Frank Hammond
William Bramley
Eric Linklater
Margaret McNamara
John Mongan
Howard S. Becker
Pauline Campanelli
Jorie Graham
Stewart Wieck
John Pilger
Steve Thayer
George Takei
Annie Leibovitz
Ben Bradlee
Petr Horáček
Jessie Redmon Fauset
H.R.F. Keating
Carmen Carter
Daniel K. Richter
Igor Stravinsky
A.N. Wilson
Jean Raspail
Steve Matchett
Paul J. Steinhardt
Minetarō Mochizuki
Duncan Bannatyne
Lynne Olson
Tom Baker
Caroline Kennedy
Nnedi Okorafor
John D. Barrow
John E. Miller
Arkady Babchenko
Tom Waits
Luis Fernando Verissimo
Jane Addams
Josef Breuer
Joel Coen
Peter D. Schiff
Charles Vess
Dan Jenkins
Robert Lee Brewer
William Blinn
Linda Robinson
Travis S. Taylor
David A. Yallop
Derek Prince
Jane  Christmas
Philip K. Howard
Isobel Kuhn

,Date,Name,Country,Year,Month,Day,description,image,match_movie1,match_movie1_descr,match_movie2,match_movie2_descr,match_movie3,match_movie3_descr,match_book1,match_book1_descr,match_book2,match_book2_descr,match_book3,match_book3_descr
0,0009-09-01,Battle of the Teutoburg Forest,NaN,9,9,1,"Allied Germanic peoples, possibly including t...",http://commons.wikimedia.org/wiki/Special:File...,Lo sbarco di Anzio,Anzio: one of WWII&apos;s bloodiest battles as...,The Lost Battalion,Fact-based war drama about an American battali...,Castle Keep,"During the Battle of the Bulge, an anachronist...","Armageddon: The Battle for Germany, 1944-1945",In September 1944 the Allies believed that H...,The Punic Wars,An impressive new historian of Roman warfare ...,A History of Warfare,The acclaimed author of The Face of Battle ex...
1,0015-01-01,Battle of the Bridges,NaN,15,1,1,Coalition intervention Naval operations Air c...,http://commons.wikimedia.org/wiki/Special:File...,A Bridge Too Far,"Operation Market Garden, September 1944: The A...",Guadalcanal Diary,The story of a large U.S. Marines invasion tas...,Hurricane,The exploits of 303 Squadron RAF during the Ba...,A Bridge Too Far,The classic account of one of the most dramat...,Miracle at Midway,The Battle of Midway was perhaps the most sig...,The Last Stand Of The Tin Can Sailors,On the morning of October 25 1944 a pivotal...
2,0016-01-01,Battle of the Weser River,NaN,16,1,1,Gallic Wars (58 BC – 57 BC) Clades Lolliana (1...,http://commons.wikimedia.org/wiki/Special:File...,Astérix &amp; Obélix: Au service de sa Majesté,Asterix crosses the channel to help second-cou...,Caesar and Cleopatra,"At the height of the Roman Civil War, a young ...",The Last Legion,"As the Roman empire crumbles, young Romulus Au...","The History of Rome, Books 1-5: The Rise of Rome",The fates ordained the founding of this gre...,The Punic Wars,An impressive new historian of Roman warfare ...,"The History of Rome, Books 1-5: The Early Hist...",Livy c 59 BC AD 17 dedicated most of his l...
3,0030-04-07,Pilate's court,NaN,30,4,7,"In the canonical gospels, Pilate's court refer...",http://commons.wikimedia.org/wiki/Special:File...,The Robe,In the Roman province of Judea during the 1st ...,The Passion of the Christ,Depicts the final twelve hours in the life of ...,Barabbas,Governor Pontius Pilate gave the populace a ch...,Works of Josephus,Encounter the world and traditions that Jesu...,The Works of Josephus,Encounter the world and traditions that Jesu...,The Last Week: A Day-by-Day Account of Jesus's...,Top Jesus scholars Marcus J Borg and John Do...
4,0043-01-01,Battle of the Medway,NaN,43,1,1,"The Battle of the Medway took place in 43 AD,...",http://commons.wikimedia.org/wiki/Special:File...,Carry on Cleo,"Two Britons, Hengist and Horsa, are captured a...",Astérix le Gaulois,"Asterix, the most cunning warrior of a Gaulish...",Outlaw King,A true David vs. Goliath story of how the 14th...,The Eagle and the Raven,Spanning three generations this historical n...,When the Eagle Hunts,Well paced and intricately plotted When th...,Scipio Africanus: Greater than Napoleon,Scipio Africanus 236 183 B C was one of th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48250,1942-11-29 00:00:00,Michael Craze,NaN,1942,11,29,birthday,NaN,Satan&apos;s Slave,A woman traveling with her parents to her uncl...,None,None,None,None,None,None,None,None,None,None
48255,1923-11-26 00:00:00,Patricia Phoenix,NaN,1923,11,26,birthday,NaN,The L-Shaped Room,"A single, pregnant woman moves into a London b...",None,None,None,None,None,None,None,None,None,None
48264,1973-07-08 00:00:00,Sally Lindsay,NaN,1973,7,8,birthday,NaN,A Matter of Loaf and Death,"Wallace and his dog, Gromit, open a bakery and...",None,None,None,None,None,None,None,None,None,None
48267,1951-07-29 00:00:00,Jack Blessing,NaN,1951,7,29,birthday,NaN,ParaNorman,"A misunderstood boy takes on ghosts, zombies a...",Missing Link,Mr. Link recruits explorer S

In [27]:
### spliting the files for git and app

# Read the all_matches DataFrame from the CSV file
all_matches = pd.read_csv("all_matches.csv")
all_matches2 = pd.read_csv("all_matches2.csv")

# Calculate the midpoint of the DataFrame
#midpoint = len(all_matches) // 5
midpoint2 = len(all_matches2) // 6

# Split the DataFrame into two halves
all_matches_part1 = all_matches.iloc[:3400]
all_matches_part2 = all_matches.iloc[3400:]

all_matches2_part1 = all_matches2.iloc[:midpoint]
all_matches2_part2 = all_matches2.iloc[midpoint:]

# Save the two smaller DataFrames to separate CSV files
all_matches_part1.to_csv("all_matches_part1.csv", index=False)
all_matches_part2.to_csv("all_matches_part2.csv", index=False)

all_matches2_part1.to_csv("all_matches2_part1.csv", index=False)
all_matches2_part2.to_csv("all_matches2_part2.csv", index=False)